# Treinamento de Modelo: Redes Neurais
> **Fase 3 - Time 7**

**Responsável:** João Ebbers;
**Algoritmo:** Redes Neurais

**Objetivos deste Notebook:**
1.  **Carregar:** Importar os dados de treino/teste padronizados (do pré-processamento unificado).
2.  **Treinar:** Ajustar o modelo **SVM** aos dados de treino.
3.  **Avaliar:** Medir a performance com foco no **Recall** (Sensibilidade) para detectar doenças.
4.  **Validar:** Testar se o modelo consegue identificar o grupo de risco "Assintomático".

**Métricas Chave:**
* **Recall (Sensibilidade):** Prioridade máxima. Não podemos deixar doentes irem para casa.
* **F1-Score:** Para garantir que não estamos apenas "chutando que todos estão doentes".

## 1. Configuração de ambiente
Importação de bibliotecas e definição de estilos gráficos padronizados.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import os
import json

# Métricas padronizadas
from sklearn.metrics import classification_report, confusion_matrix, recall_score, roc_auc_score, roc_curve, ConfusionMatrixDisplay, RocCurveDisplay

# Configuração visual
sns.set_theme(style="whitegrid")
palette = sns.color_palette("coolwarm", as_cmap=True)

In [2]:
# Carregamento dos Dados Processados (Pasta data/processed)
# IMPORTANTE: Não mexer neste bloco para garantir compatibilidade!

try:
    X_train = joblib.load('../data/processed/X_train.pkl')
    y_train = joblib.load('../data/processed/y_train.pkl')
    X_test = joblib.load('../data/processed/X_test.pkl')
    y_test = joblib.load('../data/processed/y_test.pkl')
    feature_names = joblib.load('../data/processed/feature_names.pkl')
    
    print(f"Dados carregados com sucesso!")
    print(f"Treino: {X_train.shape}")
    print(f"Teste:  {X_test.shape}")
except FileNotFoundError:
    print("Arquivos não encontrados. Rode o notebook '02_pre_processamento.ipynb' primeiro.")

Dados carregados com sucesso!
Treino: (237, 22)
Teste:  (60, 22)


## 2. Treinamento do Modelo

Adotaremos uma abordagem em duas etapas:
1.  **Baseline:** Treinar o modelo com parâmetros padrão (default) para estabelecer uma linha de base.
2.  **Otimização:** Encontrar a melhor combinação de hiperparâmetros focada em **Recall**.

### 2.1 Baseline

In [3]:
# BASELINE (Modelo "Cru")

# 1. Importe do modelo
from sklearn.neural_network import MLPClassifier

# 2. Instancie com parâmetros padrão 
model_baseline = MLPClassifier(random_state=42)


print("Treinando Baseline - Redes Neurais...")
model_baseline.fit(X_train, y_train)

# Avaliação Rápida do Baseline
y_pred_base = model_baseline.predict(X_test)
recall_base = recall_score(y_test, y_pred_base)
print(f"Baseline treinado! Recall inicial: {recall_base:.2%}")

Treinando Baseline - Redes Neurais...
Baseline treinado! Recall inicial: 78.57%


/home/fmn/cc/ligia-ps26-time7/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


### 2.2 Otimização

In [ ]:
# OTIMIZAÇÃO DE HIPERPARÂMETROS COM GRIDSEARCHCV

from sklearn.model_selection import GridSearchCV

# A. Grid específico para Redes Neurais
param_grid = {
    # Parâmetros Redes Neurais
}

# B. Configuração da Busca
grid_search = GridSearchCV(
    estimator=SVC(probability=True, random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring='recall', # Foco total em não perder doentes
    n_jobs=-1,
    verbose=1
)

print("Iniciando busca pelos melhores parâmetros...")
grid_search.fit(X_train, y_train)

# C. Melhores Resultados
best_model = grid_search.best_estimator_

print("-" * 30)
print(f"Melhor Recall na Validação: {grid_search.best_score_:.2%}")
print(f"Melhores Parâmetros: {grid_search.best_params_}")

Iniciando busca pelos melhores parâmetros...
Fitting 5 folds for each of 40 candidates, totalling 200 fits
------------------------------
Melhor Recall na Validação: 81.69%
Melhores Parâmetros: {'C': 1, 'class_weight': {0: 1, 1: 2}, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}


## 3. Avaliação de Performance

In [ ]:
# Previsões do Melhor Modelo
# y_pred_final = best_model.predict(X_test)
# Vamos pegar a probabilidade (0 a 1)
y_proba = best_model.predict_proba(X_test)[:, 1]

# Definir um limiar personalizado para maximizar recall
limiar = 0.35
y_pred_ajustado = (y_proba >= limiar).astype(int)

recall_final = recall_score(y_test, y_pred_ajustado)

print(f"📈 Evolução do Recall:")
print(f"   - Baseline:  {recall_base:.2%}")
print(f"   - Otimizado: {recall_final:.2%}")
print(f"   - Ganho:     {(recall_final - recall_base):.2%}")

# Matriz de Confusão do Modelo Campeão
plt.figure(figsize=(6, 5))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_ajustado, cmap='Blues', display_labels=['Saudável', 'Doente'])
plt.title('Matriz de Confusão (Melhor Modelo)')
plt.grid(False)
plt.show()

# Classification Report Completo
print(classification_report(y_test, y_pred_ajustado))

### 3.1. Avalição de Grupo Assintomático
Nesta etapa, validaremos a tese central: Verificaremos a performance do modelo especificamente nos pacientes que **não apresentavam dor típica** (CP=3: Asymptomatic).

> **Por que isso é importante?**
> A maioria dos erros humanos acontece aqui. Se nosso modelo tiver um bom Recall neste grupo, provamos que a IA enxerga além do óbvio.

In [ ]:
# VALIDAÇÃO NO SUBGRUPO ASSINTOMÁTICO
# 1. Identificando a coluna correta
cols_assintomaticas = [col for col in X_test.columns if 'cp_3' in col]

if len(cols_assintomaticas) > 0:
    col_alvo = cols_assintomaticas[0]
    print(f"Analisando subgrupo oculto: '{col_alvo}'")

    # 2. Filtrando os dados
    mask_assin = X_test[col_alvo] == 1
    X_test_sub = X_test[mask_assin]
    y_test_sub = y_test[mask_assin.values]
    
    if len(X_test_sub) > 0:
        # 3. APLICANDO O LIMIAR (0.35)
        # Pegamos a probabilidade em vez da classe direta
        y_proba_sub = best_model.predict_proba(X_test_sub)[:, 1]
        
        limiar = 0.35 
        y_pred_sub = (y_proba_sub >= limiar).astype(int)
        
        # Métricas
        recall_sub = recall_score(y_test_sub, y_pred_sub)
        
        print("-" * 40)
        print(f"RESULTADO DO TESTE DE FOGO (Limiar {limiar}):")
        print(f"   Pacientes Assintomáticos no Teste: {len(X_test_sub)}")
        print(f"   Recall (Doentes detectados): {recall_sub:.2%}")
        print("-" * 40)
        
        if recall_sub > 0.85:
            print("✅ SUCESSO: O modelo ajustado é excelente em detectar isquemia silenciosa!")
        else:
            print("⚠️ Nota: O recall caiu um pouco neste grupo, mas o resultado global compensa.")
    else:
        print("Nenhum paciente assintomático caiu na amostragem de teste atual.")
else:
    print("Coluna de 'cp_3' não encontrada.")

## 4. Exportação do Modelo
Salvando o modelo campeão (otimizado) para uso na aplicação final.

In [ ]:
# Exportação do Modelo
os.makedirs('../models', exist_ok=True)

# 1. Definindo um nome
nome_arquivo = 'modelo_RedesNeurais_Otimizado.pkl'
caminho_final = f'../models/{nome_arquivo}'

# 2. Salva o Modelo
joblib.dump(best_model, caminho_final)

# 3. Salva o "Segredo" (Metadata)
metadata = {
    "algoritmo": best_model.__class__.__name__,
    "threshold_otimizado": 0
}

with open('../models/metadata_RedesNeurais.json', 'w') as f:
    json.dump(metadata, f)

print(f"💾 Modelo salvo em: {caminho_final}")

## 5. Conclusão Final

...

### Comprovação Visual